<img src="images/pandas_logo.png" align="right" width="40%">

DataFrame
==========

In the last section we manipulated CSV files in parallel by building dask graphs by hand and running them with `dask` `get` functions. 

In this section we use `dask.dataframe` to build and execute dask graphs to process large volumes of CSV files automatically.

The `dask.dataframe` module implements a blocked parallel `DataFrame` object that mimics a subset of the Pandas `DataFrame`. One dask `DataFrame` is comprised of several in-memory pandas `DataFrames` separated along the index. One operation on a dask `DataFrame` triggers many pandas operations on the constituent pandas `DataFrame`s in a way that is mindful of potential parallelism and memory constraints.

**Related Documentation**

*  [Dask DataFrame documentation](http://dask.pydata.org/en/latest/dataframe.html)
*  [Pandas documentation](http://pandas.pydata.org/)

### Setup

We create artifical data.

In [ ]:
from prep import accounts_csvs
accounts_csvs(3, 1000000, 500)

import os
filename = os.path.join('data', 'accounts.*.csv')

## Load Data from CSVs and inspect the dask graph

In the last section we manually built dask graphs to read in many CSV files at once and compute their total length.  In this section we'll use `dask.dataframe` to accomplish the same result using a more Pandas-like interface rather than by playing with dictionaries manually.

### `dask.dataframe.read_csv`

This works just like `pandas.read_csv`, except on multiple csv files at once.

In [ ]:
filename

In [ ]:
import dask.dataframe as dd
df = dd.read_csv(filename)
%time len(df)

### Exercise: Inspect dask graph

Dask `DataFrame` copies a subset of the Pandas API.  

However unlike Pandas, operations on dask.dataframes don't trigger immediate computation, instead they add key-value pairs to an underlying dask graph.

In [ ]:
df._visualize()

In [ ]:
df.amount.sum()._visualize()

Above we see graphs corresponding to a call to `dd.read_csv` and `df.amount.sum()` on the result.  

Below we see the resulting computations as dictionaries.  You'll note that these dictionaries are a bit more complex than what we built by hand in the last section.  However if you look closely then you'll see all of the familiar elements of `pd.read_csv` and the filenames.

Try changing around the expression `df.amount.sum()` and see how the dictionary and graph change.  Explore a bit with the Pandas syntax that you already know.

In [ ]:
df.dask  # .dask attribute contains underlying graph

In [ ]:
df._visualize()

In [ ]:
df.amount.sum().dask

### How does this compare to Pandas?

#### Features and Size

Pandas is more mature and fully featured than `dask.dataframe`.  If your data fits in memory then you should use Pandas.  The `dask.dataframe` module gives you a limited `pandas` experience when you operate on datasets that don't fit comfortably in memory.

During this tutorial we provide a small dataset consisting of a few CSV files.  This dataset is 45MB on disk that expands to about 400MB in memory.  This dataset is small enough that you would normally use Pandas.

We've chosen this size so that exercises finish quickly.  Dask.dataframe only really becomes meaningful for problems significantly larger than this, when Pandas breaks with the dreaded 

    MemoryError:  ...
    
#### Speed

Dask.dataframe operations use several `pandas` operations internally.  Generally they run at about the same speed except in the following two cases:

1.  Dask introduces a bit of overhead, around 1ms per task.  This is usually negligible.
2.  When Pandas releases the GIL (coming to `groupby` in the next version) `dask.dataframe` can call several pandas operations in parallel increasing speed somewhat proportional to the number of cores.

Exercise: Recall and use Pandas API
----------------------------------------

If you are already familiar with the Pandas API then you should have a firm grasp on how to use `dask.dataframe`.  There are a couple of small changes.

As noted above, computations on dask `DataFrame` objects don't perform work, instead they build up a dask graph.  We can evaluate this dask graph at any time using the `.compute()` method.

In [ ]:
result = df.amount.mean()  # create lazily evaluated result
result

In [ ]:
result.compute()           # perform actual computation

Try the following exercises

1.  Use the `head()` method to get the first ten rows
2.  Use the `drop_duplicates()` method to find all of the distinct names
3.  Use selections `df[...]` to find how many positive and negative amounts there are
4.  Use groupby `df.groupby(df.A).B.func()` to get the average amount per user ID
5.  Sort the result to (4) by amount, find the names of the top 10 

This section should be easy if you are familiar with Pandas.  If you aren't then that's ok too.  You may find the [pandas documenation](http://pandas.pydata.org/) a useful read in the future.  Don't worry, future sections in this tutorial will not depend on this knowledge.

In [ ]:
# 1. Use the `head()` method to get the first ten rows
#    Note, head computes by default, this is the only operation that doesn't need an explicit call to .compute()
df.head()

In [ ]:
# 2. Use the `drop_duplicates()` method to find all of the distinct names


In [ ]:
# 3. Use selections `df[...]` to find how many positive and negative amounts there are


In [ ]:
# 3. Use selections `df[...]` to find how many positive and negative amounts there are


In [ ]:
# 4. Use groupby `df.groupby(df.A).B.func()` to get the average amount per user ID 


In [ ]:
# 5. Combine your answers to 3 and 4 to compute the average withdrawal (negative amount) per name


In [ ]:
# Solution
%load solutions/DataFrame-01.py